In [5]:
# !pip install colivara-py
# !pip install python-dotenv
from colivara_py import Colivara
from dotenv import dotenv_values


In [7]:
api_key = dotenv_values(".env")["COLIVARA_KEY"]

In [8]:
client = Colivara(api_key=api_key)

In [10]:
document = client.upsert_document(
    name='test',
    document_path='Print PC - PC Builder - Star Tech.pdf',
    wait = True
)

In [ ]:
results = client.search('tell me about the CN 700', collection_name='default_collection')
results

QueryOut(query='tell me about the CN 700', results=[PageOutQuery(collection_name='default_collection', collection_id=345, collection_metadata={}, document_name='test', document_id=15454, document_metadata={}, page_number=1, raw_score=11.187130153179169, normalized_score=0.5327204834847223, img_base64='...'), PageOutQuery(collection_name='default_collection', collection_id=345, collection_metadata={}, document_name='test', document_id=15454, document_metadata={}, page_number=2, raw_score=7.2491700649261475, normalized_score=0.34519857452029273, img_base64='...')])

In [18]:
print(results.results[0])

collection_name='default_collection' collection_id=345 collection_metadata={} document_name='test' document_id=15454 document_metadata={} page_number=1 raw_score=11.187130153179169 normalized_score=0.5327204834847223 img_base64='...'


In [20]:
dir(results.results[0])

['_PageOutQuery__properties',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer_

In [22]:
def run_rag_pipeline(query):
    # clean the query that came from the LLM
    if "not applicable" in query.lower():
        # don't need to run RAG or get a context
        return []
    if "rag query:" in query.lower():
        query = query.split("ry:")[1].strip()
    
    if "<reasoning>" in query:
        query = query.split("<reasoning>")[0].strip()

    print(f"This what we will send to the RAG pipeline: {query}")
    results = client.search(query=query, collection_name="demo collection")
    results = results.results
    # example result: 
    """{
  "query": "string",
  "results": [
    {
      "collection_name": "string",
      "collection_id": 0,
      "collection_metadata": {},
      "document_name": "string",
      "document_id": 0,
      "document_metadata": {},
      "page_number": 0,
      "raw_score": 0,
      "normalized_score": 0,
      "img_base64": "string"
    }
    ]
    }"""

    context= []
    for result in results:
        document_title = result.document_name
        page_num = result.page_number
        base64 = result.img_base64
        # base64 doesn't have data: part so we need to add it
        if "data:image" not in base64:
            base64 = f"data:image/png;base64,{base64}"
        context.append(
            {
                "metadata": f"{document_title} - Page {page_num}",
                "base64": base64,
            }
        )
    return context

query = "RAG Query: what's the cpu specification"

context = run_rag_pipeline(query)

This what we will send to the RAG pipeline: what's the cpu specification


In [23]:
context

[]